In [1]:
import numpy as np
import pandas as pd
from twitter_nlp_toolkit.tweet_sentiment_classifier import tweet_sentiment_classifier
from twitter_nlp_toolkit.file_fetcher import file_fetcher
from zipfile import ZipFile
import os

Using this library: https://github.com/eschibli/twitter-toolbox

In [8]:
df = pd.read_csv('./datasets/stacked_clean.csv')
df.shape

(1130, 18)

In [9]:
df.head()

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link,day,hour,minute,during_fire,text_nourl,key_score,from_locations
0,NaN,"Paradise, CA",1.057787e+18,Chico CA Wed Oct 31st PM Forecast: TONIGHT Mos...,2018-11-01 00:10:34+00:00,NaN,_ChicoCA,NaN,0,0,https://twitter.com/_ChicoCA/status/1057786941...,1,0,10,0,Chico CA Wed Oct 31st PM Forecast: TONIGHT Mos...,0,"Chico, CA"
1,NaN,"Paradise, CA",1.057787e+18,Happy Halloween from Harry Potter and Danaerys...,2018-11-01 00:12:17+00:00,NaN,itoddsmama,NaN,0,0,https://twitter.com/itoddsmama/status/10577873...,1,0,12,0,Happy Halloween from Harry Potter and Danaerys...,0,"Chico, CA"
2,NaN,"Paradise, CA",1.057788e+18,"happy halloween • @Chico, California https://w...",2018-11-01 00:12:58+00:00,NaN,hannahmariexx_,NaN,0,0,https://twitter.com/hannahmariexx_/status/1057...,1,0,12,0,"happy halloween • @Chico, California",0,"Chico, CA"
3,NaN,"Oroville, CA",1.057789e+18,Just posted a video @Ukla Studios https://www....,2018-11-01 00:18:43+00:00,NaN,UKLA_Music,NaN,0,0,https://twitter.com/UKLA_Music/status/10577889...,1,0,18,0,Just posted a video @Ukla Studios,0,"Gridley, CA"
4,NaN,"Oroville, CA",1.057789e+18,Just posted a video @Ukla Studios https://www....,2018-11-01 00:19:37+00:00,NaN,UKLA_Music,NaN,0,0,https://twitter.com/UKLA_Music/status/10577892...,1,0,19,0,Just posted a video @Ukla Studios,0,"Gridley, CA"


In [4]:
chunk = 1 # Fraction of data to train on - you can reduce for debugging for speed
model_path = '.models'

In [5]:
classifier = tweet_sentiment_classifier.SentimentAnalyzer()
classifier.load_small_ensemble()

Model model_full_bow_30k loading...
Module not found - are you sure it is imported?
Model model_bootstrap_glove_50N_200D_30k loading...
Module not found - are you sure it is imported?
Model bert_model_8B_1e-05L_20000.0T_Twitter_0finetuned_1 loading...
Module not found - are you sure it is imported?
Model model_full_glove_30N_200D_30k loading...
Module not found - are you sure it is imported?
Model model_full_glove_10N_100D loading...
Module not found - are you sure it is imported?


small_ensemble.zip: 100%|##########| 469M/469M [00:46<00:00, 10.7MB/s] 


Model model_full_bow_30k loading...
Model model_full_bow_30k loaded
Model model_bootstrap_glove_50N_200D_30k loading...
Module not found - are you sure it is imported?
Model bert_model_8B_1e-05L_20000.0T_Twitter_0finetuned_1 loading...
Module not found - are you sure it is imported?
Model model_full_glove_30N_200D_30k loading...
Module not found - are you sure it is imported?
Model model_full_glove_10N_100D loading...
Module not found - are you sure it is imported?


In [6]:
classifier.predict(['I am happy', 'I am sad', 'I am cheerful', 'I am mad'])

array([1., 0., 1., 0.])

In [10]:
df['sent'] = classifier.predict(df['text_nourl'])

In [ ]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

In [11]:
df2 = df[df['key_score']>0]
df2[['text_nourl', 'sent', 'key_score']]

,text_nourl,sent,key_score
101,#GrandeFire unofficial new fire report; detail...,1.0,1
126,#FrogFire unofficial new fire report; details ...,1.0,1
136,Paying it forward- “Friends helping Friends” f...,1.0,1
240,OMFreakingG! #realfood #chinooksalmon #kingsal...,1.0,1
244,"Fire in the feather river canyon. @Chico, Cali...",1.0,1
...,...,...,...
1088,Perspective on the Camp Fire. Hard to fathom.....,0.0,2
1097,Camp fire closure in #Oroville on Hwy 70 Both ...,0.0,1
1099,I have never felt so honored and grateful to s...,1.0,1
1100,THANK YOU calfire . . . . . #flyinghorsedesign...,1.0,1


In [12]:
df2['sent'].value_counts(normalize=True)

0.0    0.506329
1.0    0.493671
Name: sent, dtype: float64

In [13]:
df.to_csv('./datasets/stacked_sentiment.csv', index=False)